In [1]:
import numpy as np
import pandas as pd
import pickle
import random
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score

In [2]:
#Load the dataframe into a variable
billboard100 = pd.read_csv('Data/billboards_100.csv')
X = pd.read_csv('Data/top_songs.csv')
X_clustered = pd.read_csv('Data/playlists_clustered.csv')

In [3]:
billboard100

,song,artist
0,Flowers,Miley Cyrus
1,Kill Bill,SZA
2,Last Night,Morgan Wallen
3,Unholy,Sam Smith & Kim Petras
4,Creepin',"Metro Boomin, The Weeknd & 21 Savage"
...,...,...
95,Yandel 150,Yandel & Feid
96,Heart To Heart,Mac DeMarco
97,Next Thing You Know,Jordan Davis
98,Never Gonna Not Dance Again,P!nk


In [4]:
import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [5]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [6]:
user_song = str(input('What is your favorite song? '))

What is your favorite song? flawers


In [7]:
results = sp.search(q = user_song, limit = 3, market="GB")
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=flawers&type=track&market=GB&offset=0&limit=3',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5YGY8feqx7naU7z4HrwZM6'},
       'href': 'https://api.spotify.com/v1/artists/5YGY8feqx7naU7z4HrwZM6',
       'id': '5YGY8feqx7naU7z4HrwZM6',
       'name': 'Miley Cyrus',
       'type': 'artist',
       'uri': 'spotify:artist:5YGY8feqx7naU7z4HrwZM6'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/7I0tjwFtxUwBC1vgyeMAax'},
     'href': 'https://api.spotify.com/v1/albums/7I0tjwFtxUwBC1vgyeMAax',
     'id': '7I0tjwFtxUwBC1vgyeMAax',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273f429549123dbe8552764ba1d',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02f429549123dbe8552764ba1d',
       'width': 300},
      {'height': 64,
       'url': 'https://i.s

In [8]:
#Find artist
user_artist = results['tracks']['items'][0]['artists'][0]['name']

#Track ID
user_id = results['tracks']['items'][0]['id']

#Audio features
user_features = sp.audio_features(tracks=[user_id])

In [9]:
user_song_df = pd.DataFrame(user_features)
user_song_df = user_song_df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo"]]
user_song_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.707,0.681,-4.325,0.0668,0.0632,0.000005,0.0322,0.646,117.999


In [10]:
X = pd.DataFrame(X, columns= ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])

print()
X.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.514,0.730,-5.934,0.0598,0.00146,0.000095,0.0897,0.334,171.005
1,0.707,0.681,-4.325,0.0668,0.06320,0.000005,0.0322,0.646,117.999
2,0.644,0.735,-5.747,0.0391,0.05210,0.144000,0.1610,0.418,88.980
3,0.715,0.620,-6.005,0.0484,0.41700,0.000000,0.0822,0.172,97.950
4,0.440,0.317,-9.258,0.0531,0.89100,0.000005,0.1410,0.268,169.914


In [11]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
display(X.head())
print()
display(X_scaled_df.head())

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.514,0.730,-5.934,0.0598,0.00146,0.000095,0.0897,0.334,171.005
1,0.707,0.681,-4.325,0.0668,0.06320,0.000005,0.0322,0.646,117.999
2,0.644,0.735,-5.747,0.0391,0.05210,0.144000,0.1610,0.418,88.980
3,0.715,0.620,-6.005,0.0484,0.41700,0.000000,0.0822,0.172,97.950
4,0.440,0.317,-9.258,0.0531,0.89100,0.000005,0.1410,0.268,169.914


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-0.951795,0.415223,0.481467,-0.191236,-0.895051,-0.235272,-0.583126,-0.997710,1.827186
1,0.389657,0.150565,0.948845,-0.094941,-0.628441,-0.236089,-1.020810,0.271709,-0.119582
2,-0.048226,0.442229,0.535786,-0.475992,-0.676374,1.067969,-0.040398,-0.655944,-1.185372
3,0.445261,-0.178909,0.460843,-0.348058,0.899362,-0.236136,-0.640215,-1.656832,-0.855928
4,-1.466134,-1.815472,-0.484080,-0.283403,2.946220,-0.236093,-0.192636,-1.266241,1.787116


In [12]:
kmeans = KMeans(n_clusters = 7, random_state = 1234)
kmeans.fit(X_scaled_df)

KMeans(n_clusters=7, random_state=1234)

In [13]:
kmeans.labels_
X

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.514,0.730,-5.934,0.0598,0.00146,0.000095,0.0897,0.3340,171.005
1,0.707,0.681,-4.325,0.0668,0.06320,0.000005,0.0322,0.6460,117.999
2,0.644,0.735,-5.747,0.0391,0.05210,0.144000,0.1610,0.4180,88.980
3,0.715,0.620,-6.005,0.0484,0.41700,0.000000,0.0822,0.1720,97.950
4,0.440,0.317,-9.258,0.0531,0.89100,0.000005,0.1410,0.2680,169.914
...,...,...,...,...,...,...,...,...,...
548,0.692,0.525,-12.691,0.0373,0.02120,0.001900,0.0944,0.0879,103.679
549,0.712,0.640,-14.215,0.0321,0.01110,0.047800,0.0638,0.9250,130.259
550,0.716,0.860,-7.807,0.0460,0.00917,0.839000,0.2510,0.9580,142.168
551,0.524,0.330,-14.264,0.0297,0.65400,0.000000,0.0635,0.5530,179.598


In [14]:
user_scaled = scaler.transform(user_song_df)
user_scaled_df = pd.DataFrame(user_scaled, columns = user_song_df.columns)
user_scaled_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.389657,0.150565,0.948845,-0.094941,-0.628441,-0.236089,-1.02081,0.271709,-0.119582


In [15]:
# Get the cluster label of user song
user_cluster = kmeans.predict(user_scaled_df)

user_cluster_num = user_cluster[0]

In [16]:
X_clustered = pd.read_csv('data/playlists_clustered.csv')

In [17]:
X_clustered

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,artist,name,album,popularity,cluster
0,0.514,0.730,-5.934,0.0598,0.00146,0.000095,0.0897,0.3340,171.005,0VjIjW4GlUZAMYd2vXMi3b,200040,The Weeknd,Blinding Lights,After Hours,87,5
1,0.707,0.681,-4.325,0.0668,0.06320,0.000005,0.0322,0.6460,117.999,0yLdNVWF3Srea0uzk55zFn,200455,Miley Cyrus,Flowers,Flowers,100,2
2,0.644,0.735,-5.747,0.0391,0.05210,0.144000,0.1610,0.4180,88.980,1Qrg8KqiBpW07V7PNxwwwL,153947,SZA,Kill Bill,SOS,91,2
3,0.715,0.620,-6.005,0.0484,0.41700,0.000000,0.0822,0.1720,97.950,2dHHgzDwk4BJdRwy9uXhTO,221520,Metro Boomin,Creepin' (with The Weeknd & 21 Savage),HEROES & VILLAINS,93,4
4,0.440,0.317,-9.258,0.0531,0.89100,0.000005,0.1410,0.2680,169.914,6xGruZOHLs39ZbVccQTuPZ,233456,Joji,Glimpse of Us,Glimpse of Us,84,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,0.692,0.525,-12.691,0.0373,0.02120,0.001900,0.0944,0.0879,103.679,0vZCwj0Mq5uS8GwSIYo6Y7,333200,Rod Stewart,Passion,Foolish Behaviour,39,4
549,0.712,0.640,-14.215,0.0321,0.01110,0.047800,0.0638,0.9250,130.259,1PuUboooNpZFmtsCJRF4sn,252067,Liquid Gold,Dance Yourself Dizzy,Liquid Gold,39,4
550,0.716,0.860,-7.807,0.0460,0.00917,0.839000,0.2510,0.9580,142.168,2CfqGS6D9piBWM3SAjbUJM,201013,Spandau Ballet,To Cut A Long Story Short,Gold - The Best of Spandau Ballet,38,6
551,0.524,0.330,-14.264,0.0297,0.65400,0.000000,0.0635,0.5530,179.598,7mZBgeTb3oLqTRwC1suWXN,255000,Peaches & Herb,I Pledge My Love,At Their Best,35,3


In [18]:
df = X_clustered[X_clustered['cluster'] == user_cluster_num]

In [19]:
df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,artist,name,album,popularity,cluster
1,0.707,0.681,-4.325,0.0668,0.0632,0.000005,0.0322,0.646,117.999,0yLdNVWF3Srea0uzk55zFn,200455,Miley Cyrus,Flowers,Flowers,100,2
2,0.644,0.735,-5.747,0.0391,0.0521,0.144000,0.1610,0.418,88.980,1Qrg8KqiBpW07V7PNxwwwL,153947,SZA,Kill Bill,SOS,91,2
7,0.637,0.643,-6.571,0.0519,0.1300,0.000002,0.1420,0.533,97.008,0V3wPSX9ygBnCm8psDIegu,200690,Taylor Swift,Anti-Hero,Midnights,91,2
10,0.788,0.859,-2.724,0.0856,0.2810,0.000000,0.0424,0.822,141.020,50nfwKoDiSYg8zOCREWAm5,207853,Ed Sheeran,Shivers,=,83,2
14,0.538,0.742,-5.355,0.1140,0.1380,0.000047,0.0934,0.250,96.107,5WxVXxCMRnvxUKFq40ELwq,272373,RAYE,Escapism.,Escapism. (Sped Up),79,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,0.817,0.767,-7.194,0.0468,0.4570,0.072300,0.0606,0.878,128.930,4Ku6X72Q815qNd7ChuiDdD,231880,Paul McCartney,Coming Up - Remastered 2011,McCartney II (Special Edition),48,2
527,0.778,0.667,-7.273,0.0258,0.0514,0.031200,0.0920,0.964,96.954,165NAtd2DRPKNQ6EVY0VG9,233893,Blondie,The Tide Is High - Edit,Tide Is High,48,2
530,0.837,0.703,-9.385,0.0415,0.4150,0.000027,0.1530,0.914,121.556,1wouF9mtyiwZDYiq5YElaq,245480,The Spinners,"Working My Way Back to You / Forgive Me, Girl",Smooth And Sweet,48,2
532,0.683,0.774,-6.736,0.0337,0.4830,0.000016,0.0673,0.742,113.150,1Urr1PLp5Cy0ylztRSJvdB,328267,Stephanie Mills,Never Knew Love Like This Before (Extended Mix),"Groove On, Vol. 1",47,2


In [20]:
joined = pd.merge(df, X)

In [21]:
joined

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,artist,name,album,popularity,cluster
0,0.707,0.681,-4.325,0.0668,0.0632,0.000005,0.0322,0.646,117.999,0yLdNVWF3Srea0uzk55zFn,200455,Miley Cyrus,Flowers,Flowers,100,2
1,0.644,0.735,-5.747,0.0391,0.0521,0.144000,0.1610,0.418,88.980,1Qrg8KqiBpW07V7PNxwwwL,153947,SZA,Kill Bill,SOS,91,2
2,0.637,0.643,-6.571,0.0519,0.1300,0.000002,0.1420,0.533,97.008,0V3wPSX9ygBnCm8psDIegu,200690,Taylor Swift,Anti-Hero,Midnights,91,2
3,0.788,0.859,-2.724,0.0856,0.2810,0.000000,0.0424,0.822,141.020,50nfwKoDiSYg8zOCREWAm5,207853,Ed Sheeran,Shivers,=,83,2
4,0.538,0.742,-5.355,0.1140,0.1380,0.000047,0.0934,0.250,96.107,5WxVXxCMRnvxUKFq40ELwq,272373,RAYE,Escapism.,Escapism. (Sped Up),79,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,0.817,0.767,-7.194,0.0468,0.4570,0.072300,0.0606,0.878,128.930,4Ku6X72Q815qNd7ChuiDdD,231880,Paul McCartney,Coming Up - Remastered 2011,McCartney II (Special Edition),48,2
171,0.778,0.667,-7.273,0.0258,0.0514,0.031200,0.0920,0.964,96.954,165NAtd2DRPKNQ6EVY0VG9,233893,Blondie,The Tide Is High - Edit,Tide Is High,48,2
172,0.837,0.703,-9.385,0.0415,0.4150,0.000027,0.1530,0.914,121.556,1wouF9mtyiwZDYiq5YElaq,245480,The Spinners,"Working My Way Back to You / Forgive Me, Girl",Smooth And Sweet,48,2
173,0.683,0.774,-6.736,0.0337,0.4830,0.000016,0.0673,0.742,113.150,1Urr1PLp5Cy0ylztRSJvdB,328267,Stephanie Mills,Never Knew Love Like This Before (Extended Mix),"Groove On, Vol. 1",47,2


In [22]:
def recommender():
    random_row = []
    if user_song in billboard100['song'].values:
        random_row = billboard100.sample(n=1)
        print('Your favrite song is at top 100 right now, I recommend you to check this hot song as well: ')
        print(random_row['song'] + ' by ' + random_row['artist'])
    else:
        random_song = df.sample(n=1)
        random_row = joined.sample(n=1)
        x = random_row['id']
        print('Here is my recommendation for you based on your taste: ')
        print('open.spotify.com/track/' + x)

    

In [23]:
recommender()

Here is my recommendation for you based on your taste: 
35    open.spotify.com/track/0U10zFw4GlBacOy9VDGfGL
Name: id, dtype: object
